## Tacotron 2 inference code 
Edit the variables **checkpoint_path** and **text** to match yours and run the entire code to generate plots of mel outputs, alignments and audio synthesis from the generated mel-spectrogram using Griffin-Lim.

#### Import libraries and setup matplotlib

In [1]:
import matplotlib
%matplotlib inline
import matplotlib.pylab as plt

import IPython.display as ipd

import sys
sys.path.append('waveglow/')
import numpy as np
import torch

from hparams import create_hparams
from model import Tacotron2
from layers import TacotronSTFT, STFT
from audio_processing import griffin_lim
from train import load_model
from text import text_to_sequence
from denoiser import Denoiser

def plot_data(data, figsize=(16, 4)):
    fig, axes = plt.subplots(1, len(data), figsize=figsize)
    for i in range(len(data)):
        axes[i].imshow(data[i], aspect='auto', origin='lower', 
                       interpolation='none')

# waveglow
waveglow_path = 'mdl_pretrain/waveglow_256channels_universal_v5.pt'
waveglow = torch.load(waveglow_path)['model']
waveglow.cuda().eval()
for k in waveglow.convinv:
    k.float()
denoiser = Denoiser(waveglow)

# param setup
hparams = create_hparams()
hparams.sampling_rate = 22050

/tmp/ipykernel_86540/520630016.py:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  waveglow = torch.load(waveglow_path)['model']
/home/nanxun/anaconda3/envs/audio/lib/pytho

#### Prepare text input

In [3]:
text = "nu2 er3 bie2 er3 de2 ye1 wa2 qing3 zhou1 yong3 kang1 zhuan3 da2 dui4 wu2 bang1 guo2 wei3 yuan2 zhang3 de5 qin1 qie4 wen4 hou4."
sequence = np.array(text_to_sequence(text, ['basic_cleaners']))[None, :]
sequence = torch.autograd.Variable(
    torch.from_numpy(sequence)).cuda().long()
print(sequence)

tensor([[ 51,  58, 150,  11,  42,  55, 151,  11,  39,  46,  42, 150,  11,  42,
          55, 151,  11,  41,  42, 150,  11,  62,  42, 149,  11,  60,  38, 150,
          11,  54,  46,  51,  44, 151,  11,  63,  45,  52,  58, 149,  11,  62,
          52,  51,  44, 151,  11,  48,  38,  51,  44, 149,  11,  63,  45,  58,
          38,  51, 151,  11,  41,  38, 150,  11,  41,  58,  46, 152,  11,  60,
          58, 150,  11,  39,  38,  51,  44, 149,  11,  44,  58,  52, 150,  11,
          60,  42,  46, 151,  11,  62,  58,  38,  51, 150,  11,  63,  45,  38,
          51,  44, 151,  11,  41,  42, 153,  11,  54,  46,  51, 149,  11,  54,
          46,  42, 152,  11,  60,  42,  51, 152,  11,  45,  52,  58, 152,   7]],
       device='cuda:0')


#### Load model from checkpoint

In [4]:
checkpoint_path = "mdls/checkpoint_7600"
model = load_model(hparams)
model.load_state_dict(torch.load(checkpoint_path)['state_dict'])
_ = model.cuda().eval()

/tmp/ipykernel_86540/3534768453.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(checkpoint_path)['state_dict'])


#### Decode text input and plot results

In [7]:
with torch.no_grad():
    mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)
plot_data((mel_outputs.float().data.cpu().numpy()[0],
           mel_outputs_postnet.float().data.cpu().numpy()[0],
           alignments.float().data.cpu().numpy()[0].T))

#### Synthesize audio from spectrogram using WaveGlow

In [6]:
with torch.no_grad():
    audio = waveglow.infer(mel_outputs_postnet, sigma=0.666)
ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate)

OutOfMemoryError: CUDA out of memory. Tried to allocate 314.00 MiB. GPU 0 has a total capacity of 15.70 GiB of which 290.25 MiB is free. Process 60548 has 14.11 GiB memory in use. Including non-PyTorch memory, this process has 1.27 GiB memory in use. Of the allocated memory 932.82 MiB is allocated by PyTorch, and 79.18 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

#### (Optional) Remove WaveGlow bias

In [9]:
audio_denoised = denoiser(audio, strength=0.01)[:, 0]
ipd.Audio(audio_denoised.cpu().numpy(), rate=hparams.sampling_rate) 

TypeError: pad_center() takes 1 positional argument but 2 were given